In [0]:
from google.colab import drive
drive.mount("/content/gdrive")
print("Done")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
Done


In [0]:
import matplotlib
# matplotlib.use("Agg")

# import the necessary packages
import keras
from keras.applications.vgg16 import VGG16
from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, Input, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
#from pyimagesearch.smallervggnet import SmallerVGGNet
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
from tqdm import tqdm
import argparse
import random
import pickle
import cv2
import os
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

Using TensorFlow backend.


In [0]:
EPOCHS = 10
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (128,128,3)
num_classes = 10

In [0]:
train = np.load("gdrive/My Drive/Seminar_Data.npy")
print(train.shape)

X_train = np.array([i[0] for i in train]).reshape(-1,128,128,3)
Y_train = np.array([i[1] for i in train])



del train
print("Train Loaded")
print(X_train.shape)

(3747, 2)
Train Loaded
(3747, 128, 128, 3)


In [0]:
train = np.load("gdrive/My Drive/Seminar_Data_Test.npy")
print(train.shape)

X_test = np.array([i[0] for i in train]).reshape(-1,128,128,3)
Y_test = np.array([i[1] for i in train])



del train
print("Test Loaded")
print(X_test.shape)

(919, 2)
Test Loaded
(919, 128, 128, 3)


In [0]:
print(Y_train.shape)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

X_train /= 255
X_test /= 255

Y_train = keras.utils.to_categorical(Y_train,num_classes)
Y_test = keras.utils.to_categorical(Y_test,num_classes)
print(Y_train.shape)

(3747,)
(3747, 10)


In [0]:
model = Sequential()
model.add(Conv2D(96, 3, strides=1, input_shape=(128,128,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=3, strides=2))
# model.add(Dropout(0.3))

model.add(Conv2D(256, 5, strides=1))
model.add(ZeroPadding2D(2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=3, strides=2))
# model.add(Dropout(0.3))

model.add(Conv2D(384, 3, strides=1))
model.add(ZeroPadding2D(1))
model.add(Activation('relu'))
# model.add(Dropout(0.3))

model.add(Conv2D(384, 3, strides=1))
model.add(ZeroPadding2D(1))
model.add(Activation('relu'))
# model.add(Dropout(0.3))


model.add(Conv2D(256, 3, strides=1))
model.add(ZeroPadding2D(1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=3, strides=2))
# model.add(Dropout(0.3))


model.add(Flatten())

model.add(Dense(4096))
model.add(Activation('relu'))
# model.add(Dropout(0.3))

model.add(Dense(4096))
model.add(Activation('relu'))
# model.add(Dropout(0.3))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 126, 126, 96)      2688      
_________________________________________________________________
activation_9 (Activation)    (None, 126, 126, 96)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 62, 62, 96)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 58, 58, 256)       614656    
_________________________________________________________________
zero_padding2d_5 (ZeroPaddin (None, 62, 62, 256)       0         
_________________________________________________________________
activation_10 (Activation)   (None, 62, 62, 256)       0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 30, 30, 256)       0         
__________

In [0]:
model.load_weights("gdrive/My Drive/Model_Weights/model128-006-.h5")

In [0]:
## CREATE DIRECTORY FIRST /home/cudacenter/DR/FINAL_05Feb/Pretrained/DenseNet_Weights_t1
sgd = SGD(lr=1e-3, decay=1e-6, momentum = 0.9, nesterov=True)
checkpoint = ModelCheckpoint('gdrive/My Drive/Model_Weights/model128-{epoch:03d}-.h5',verbose = 1,monitor = 'acc',save_best_only = True,mode = "auto")
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=sgd, metrics=['accuracy'])

In [0]:
image = cv2.resize(cv2.imread("gdrive/My Drive/1.jpg",cv2.IMREAD_COLOR), (128, 128))
image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
image = img_to_array(image)
image = np.array(image)
image = image.reshape(-1,128,128,3)
model.predict(image)

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [0]:
model.fit(X_train, Y_train,batch_size=BS,epochs=EPOCHS,verbose=1,validation_data=(X_test, Y_test), callbacks=[checkpoint])

Train on 3747 samples, validate on 919 samples
Epoch 1/10
3747/3747 [==============================] - 51s 13ms/step - loss: 0.0819 - acc: 0.9712 - val_loss: 0.1668 - val_acc: 0.9467

Epoch 00001: acc improved from -inf to 0.97118, saving model to gdrive/My Drive/Model_Weights/model128-001-.h5
Epoch 2/10
3747/3747 [==============================] - 47s 13ms/step - loss: 0.0799 - acc: 0.9746 - val_loss: 0.1905 - val_acc: 0.9282

Epoch 00002: acc improved from 0.97118 to 0.97465, saving model to gdrive/My Drive/Model_Weights/model128-002-.h5
Epoch 3/10
3747/3747 [==============================] - 47s 12ms/step - loss: 0.0630 - acc: 0.9784 - val_loss: 0.1576 - val_acc: 0.9510

Epoch 00003: acc improved from 0.97465 to 0.97838, saving model to gdrive/My Drive/Model_Weights/model128-003-.h5
Epoch 4/10
3747/3747 [==============================] - 47s 12ms/step - loss: 0.0517 - acc: 0.9827 - val_loss: 0.2020 - val_acc: 0.9271

Epoch 00004: acc improved from 0.97838 to 0.98265, saving model to

In [0]:
score = model.evaluate(X_test,Y_test,verbose=1)
print("Test Loss : ",score[0])
print("Test Accuracy : ",score[1])
Y_pred = model.predict(X_test,verbose = 1)

919/919 [==============================] - 3s 3ms/step
Test Loss :  0.16011752434261345
Test Accuracy :  0.9532100108813928
919/919 [==============================] - 3s 3ms/step


In [0]:
from pathlib import Path

In [0]:
# Save neural network structure
model_structure = model.to_json()
f = Path("model_structure_leaf.json")
f.write_text(model_structure)

6692

In [0]:
model.save_weights('gdrive/My Drive/Model_Weights/DR_weights_{}_modelF128.h5'.format(score[1]))

NameError: ignored